In [1]:
#Import library
import datetime
import praw
import pandas as pd
import time
import re
import uuid

In [2]:
def get_reddit_client():
    clientId = "0vJWkJK0D4GjG_UjIJptUQ"
    clientSecret = "YSryCtQqZnOBCv33sb3SJjHA3nGrag"
    userName = "tasmeer_01"
    userAgents = 'my_reddit_on_myki'
    redditClient = praw.Reddit(client_id = clientId,
                                       client_secret = clientSecret,
                                       username = userName,
                                       user_agent = userAgents)
    return redditClient


In [3]:
def collect_reddit_data(
    search_queries,
    subreddit_name='melbourne',
    time_filter='year',
    posts_limit=400,
    #comments_limit_per_post=200,
    min_comment_length=5,
    excluded_subreddits=None
):
    reddit = get_reddit_client()
    two_years_ago = int((datetime.datetime.now() - datetime.timedelta(days=730)).timestamp())

    posts    = []
    comments = []
    post_count = comment_count = 0
    excluded = set(sub.lower() for sub in (excluded_subreddits or []))

    for q in search_queries:
        print(f"Searching for: {q}")
        submissions = reddit.subreddit(subreddit_name) \
                            .search(query=q, limit=1000, sort='hot', time_filter=time_filter)
        for subm in submissions:
            #if subm.created_utc < two_years_ago:
            #    continue
            if subm.subreddit.display_name.lower() in excluded:
                continue

            # --- collect post ---
            posts.append({
                'query':        q,
                'post_id':      subm.id,
                'author_name':  subm.author.name if subm.author else '[deleted]',
                'title':        subm.title,
                'text':         subm.selftext,
                'score':        subm.score,
                'num_comments': subm.num_comments,
                'created_utc':  subm.created_utc,
                'subreddit':    subm.subreddit.display_name
            })
            post_count += 1
            if post_count >= posts_limit:
                break

            # --- collect comments ---
            subm.comments.replace_more(limit=0)
            for cm in subm.comments.list():#[:comments_limit_per_post]:
                #if len(cm.body) < min_comment_length:
                # continue
                comments.append({
                    'query':        q,
                    'post_id':      subm.id,  
                    'comment_id':   cm.id,
                    'parent_id':    cm.parent_id,  
                    'author_name':  cm.author.name if cm.author else  uuid.uuid4().hex, #'[deleted]',
                    'text':         cm.body,
                    'score':        cm.score,
                    'created_utc':  cm.created_utc,
                    'subreddit':    subm.subreddit.display_name
                })
                comment_count += 1
                if comment_count % 100 == 0:
                    print(f"Retrieved {comment_count} comments so far.")

        print(f"Update {post_count} posts, {comment_count} comments so far.")

    posts_df    = pd.DataFrame(posts)
    comments_df = pd.DataFrame(comments)

    # Save raw CSVs if you like:
    posts_df.to_csv('reddit_posts.csv', index=False)
    comments_df.to_csv('reddit_comments.csv', index=False)
    
expanded_queries = [
    "Melbourne transport Fares",
    "Myki daily cost",
    "Melbourne train Fares",
    "Myki Fares",
    "Myki fines",
    "Myki",
    "PTV Prices review"
]
collect_reddit_data(expanded_queries)

Searching for: Melbourne transport Fares
Retrieved 100 comments so far.
Retrieved 200 comments so far.
Retrieved 300 comments so far.
Retrieved 400 comments so far.
Retrieved 500 comments so far.
Retrieved 600 comments so far.
Retrieved 700 comments so far.
Retrieved 800 comments so far.
Retrieved 900 comments so far.
Retrieved 1000 comments so far.
Retrieved 1100 comments so far.
Retrieved 1200 comments so far.
Retrieved 1300 comments so far.
Retrieved 1400 comments so far.
Retrieved 1500 comments so far.
Retrieved 1600 comments so far.
Retrieved 1700 comments so far.
Retrieved 1800 comments so far.
Retrieved 1900 comments so far.
Retrieved 2000 comments so far.
Retrieved 2100 comments so far.
Retrieved 2200 comments so far.
Retrieved 2300 comments so far.
Retrieved 2400 comments so far.
Retrieved 2500 comments so far.
Retrieved 2600 comments so far.
Retrieved 2700 comments so far.
Retrieved 2800 comments so far.
Retrieved 2900 comments so far.
Retrieved 3000 comments so far.
Retrieve